In [1]:
using TextAnalysis, CorpusLoaders, MultiResolutionIterators

┌ Info: Recompiling stale cache file /home/ayushk4/.julia/compiled/v1.0/TextAnalysis/5Mwet.ji for TextAnalysis [a2db99b7-8b79-58f8-94bf-bbc811eef33d]
└ @ Base loading.jl:1190
┌ Warning: Package TextAnalysis does not have Libdl in its dependencies:
│ - If you have TextAnalysis checked out for development and have
│   added Libdl as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with TextAnalysis
└ Loading Libdl into TextAnalysis from project dependency, future warnings for TextAnalysis are suppressed.
┌ Info: Recompiling stale cache file /home/ayushk4/.julia/compiled/v1.0/CorpusLoaders/jEgYN.ji for CorpusLoaders [8f4694b6-a17a-549f-a57e-76e1fca3f017]
└ @ Base loading.jl:1190
┌ Warning: Package CorpusLoaders does not have CSV in its dependencies:
│ - If you have CorpusLoaders checked out for development and have
│   added CSV as a dependency but haven't updated your primary
│   environment'

In [2]:
pos = POS_Tagger()

TextAnalysis.POS_model{TextAnalysis.BiLSTM_CNN_CRF_Model{Flux.Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},Array{Float32,2},Flux.Recur{Flux.LSTMCell{TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}},Flux.Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},CRF{TrackedArray{…,Array{Float32,2}}},Array{Int64,2}}}(TextAnalysis.BiLSTM_CNN_CRF_Model{Flux.Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},Array{Float32,2},Flux.Recur{Flux.LSTMCell{TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}},Flux.Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},CRF{TrackedArray{…,Array{Float32,2}}},Array{Int64,2}}(["NNP", "VBZ", "JJ", "NN", "TO", "VB", ".", "CD", "DT", "VBD"  …  "NNPS", "SYM", "RBS", "UH", "PDT", "''", "LS", "JJS", "WP\$", "NN|SYM"], Dict('w'=>81,'E'=>34,'7'=>22,'Z'=>55,']'=>57,'o'=>73,'B

In [3]:
train_set = CorpusLoaders.load(CoNLL(), "train") # training set
test_set = CorpusLoaders.load(CoNLL(), "test") # test set
dev_set = CorpusLoaders.load(CoNLL(), "dev") # dev set

train_dataset = flatten_levels(train_set, lvls(CoNLL, :document)) |> full_consolidate
dev_dataset = flatten_levels(dev_set, lvls(CoNLL, :document)) |> full_consolidate
test_dataset = flatten_levels(test_set, lvls(CoNLL, :document)) |> full_consolidate

CorpusLoaders.Document{Array{Array{CorpusLoaders.NERTaggedWord,1},1},String}("test.txt", Array{CorpusLoaders.NERTaggedWord,1}[[NERTaggedWord("O", "B-NP", "NN", "SOCCER"), NERTaggedWord("O", "O", ":", "-"), NERTaggedWord("B-LOC", "B-NP", "NNP", "JAPAN"), NERTaggedWord("O", "B-VP", "VB", "GET"), NERTaggedWord("O", "B-NP", "NNP", "LUCKY"), NERTaggedWord("O", "I-NP", "NNP", "WIN"), NERTaggedWord("O", "O", ",", ","), NERTaggedWord("B-PER", "B-NP", "NNP", "CHINA"), NERTaggedWord("O", "B-PP", "IN", "IN"), NERTaggedWord("O", "B-NP", "DT", "SURPRISE"), NERTaggedWord("O", "I-NP", "NN", "DEFEAT"), NERTaggedWord("O", "O", ".", ".")], [NERTaggedWord("B-PER", "B-NP", "NNP", "Nadim"), NERTaggedWord("I-PER", "I-NP", "NNP", "Ladki")], [NERTaggedWord("B-LOC", "B-NP", "NNP", "AL-AIN"), NERTaggedWord("O", "O", ",", ","), NERTaggedWord("B-LOC", "B-NP", "NNP", "United"), NERTaggedWord("I-LOC", "I-NP", "NNP", "Arab"), NERTaggedWord("I-LOC", "I-NP", "NNPS", "Emirates"), NERTaggedWord("O", "I-NP", "CD", "1996-

In [4]:
function obtain_X_Y(dataset)
    X = [CorpusLoaders.word.(sent) for sent in dataset]
    Y = [CorpusLoaders.part_of_speech.(sent) for sent in dataset]
    return X, Y
end

X_train, Y_train = obtain_X_Y(train_dataset)
X_dev, Y_dev = obtain_X_Y(dev_dataset)
X_test, Y_test = obtain_X_Y(test_dataset)

(Array{String,1}[["SOCCER", "-", "JAPAN", "GET", "LUCKY", "WIN", ",", "CHINA", "IN", "SURPRISE", "DEFEAT", "."], ["Nadim", "Ladki"], ["AL-AIN", ",", "United", "Arab", "Emirates", "1996-12-06"], ["Japan", "began", "the", "defence", "of", "their", "Asian", "Cup", "title", "with"  …  "Syria", "in", "a", "Group", "C", "championship", "match", "on", "Friday", "."], ["But", "China", "saw", "their", "luck", "desert", "them", "in", "the", "second"  …  "crashing", "to", "a", "surprise", "2-0", "defeat", "to", "newcomers", "Uzbekistan", "."], ["China", "controlled", "most", "of", "the", "match", "and", "saw", "several", "chances"  …  "the", "advancing", "Chinese", "keeper", "and", "into", "an", "empty", "net", "."], ["Oleg", "Shatskiku", "made", "sure", "of", "the", "win", "in", "injury", "time"  …  "unstoppable", "left", "foot", "shot", "from", "just", "outside", "the", "area", "."], ["The", "former", "Soviet", "republic", "was", "playing", "in", "an", "Asian", "Cup", "finals", "tie", "for", "t

In [5]:
function eval(pos, X, Y)
    num_correct = 0
    num_total = 0
    
    for (x_seq, y_seq) in zip(X, Y)
        preds = pos(x_seq)
    
        for i in eachindex(preds)
            num_total += 1
            if preds[i] == y_seq[i]
                num_correct += 1
            end
        end
    end

    println("Accuracy: ", num_correct / num_total)
end

eval (generic function with 2 methods)

In [6]:
eval(pos, X_train, Y_train)

Accuracy: 0.9187313685720039


In [7]:
eval(pos, X_dev, Y_dev)

Accuracy: 0.9115493944939839


In [8]:
eval(pos, X_test, Y_test)

Accuracy: 0.9039517605254657


In [9]:
tagger = PerceptronTagger(true)

loaded successfully


PerceptronTagger(TextAnalysis.AveragePerceptron(Set(Any["JJS", "NNP_VBZ", "NN_NNS", "CC", "NNP_NNS", "EX", "NNP_TO", "VBD_DT", "LS", ("Council", "NNP")  …  "NNPS", "NNP_LS", "VB", "NNS_NN", "NNP_SYM", "VBZ", "VBZ_JJ", "UH", "SYM", "NNP_NN", "CD"]), Dict{Any,Any}("i+2 word wetlands"=>Dict{Any,Any}("NNS"=>0.0,"JJ"=>0.0,"NN"=>0.0),"i-1 tag+i word NNP basic"=>Dict{Any,Any}("JJ"=>0.0,"IN"=>0.0),"i-1 tag+i word DT chloride"=>Dict{Any,Any}("JJ"=>0.0,"NN"=>0.0),"i-1 tag+i word NN choo"=>Dict{Any,Any}("NNP"=>0.0,"NN"=>0.0),"i+1 word antarctica"=>Dict{Any,Any}("FW"=>0.0,"NN"=>0.0),"i-1 tag+i word -START- appendix"=>Dict{Any,Any}("NNP"=>0.0,"NNPS"=>0.0,"NN"=>0.0),"i-1 word wahoo"=>Dict{Any,Any}("JJ"=>0.0,"VBD"=>0.0),"i-1 tag+i word DT children's"=>Dict{Any,Any}("NNS"=>0.0,"NN"=>0.0),"i word dnipropetrovsk"=>Dict{Any,Any}("NNP"=>0.003,"NN"=>-0.003),"i suffix hla"=>Dict{Any,Any}("JJ"=>0.0,"NN"=>0.0)…), DataStructures.DefaultDict{Any,Any,Int64}(), DataStructures.DefaultDict{Any,Any,Int64}(), 1, ["-S

In [55]:
function eval_(tagger, X, Y)
    num_correct = 0
    num_total = 0

    for (x_seq, y_seq) in zip(X, Y)
        preds = predict(tagger, x_seq)
        for i in eachindex(preds)
            (w, y) = preds[i]
            typeof(y) == Missing && continue
            num_total += 1
            if y == y_seq[i]
                num_correct += 1
            end
        end
    end

    println("Accuracy: ", num_correct / num_total)
end

eval_ (generic function with 2 methods)

In [56]:
eval_(tagger, X_test, Y_test)

Accuracy: 0.45400094798983065


In [57]:
eval_(tagger, X_train, Y_train)

Accuracy: 0.46666568407813547


In [58]:
eval_(tagger, X_dev, Y_dev)

Accuracy: 0.4735857877512856
